In [ ]:
import pandas as pd
import pickle

In [ ]:
experiment_name = "2024.10.10-02-AnalisiSentimentConGemma"

In [ ]:
table_name = 'vinc_Gemma_sentiment_processed_ids'

In [ ]:
model_name = "google/gemma-1.1-7b-it"

In [ ]:
from google.colab import drive
from IPython.display import Image, display

mount_point = "/content/gdrive"
base_path = mount_point + "/MyDrive/Colab/SDG/data"

input_data_path = base_path + "/../AnalisiTesti/data/output/full_data___6400_texts___sentiment_sdgwashing_sdgsmearing.xlsx"

output_data_path = base_path + "/../AnalisiTesti/data/output/" + experiment_name
log_filename = base_path + "/../AnalisiTesti/logs/" + experiment_name + ".log"

drive.mount(mount_point, force_remount=True)

Mounted at /content/gdrive


In [ ]:
with open(base_path+"/../AnalisiTesti/cred.txt", 'r') as file:
    lines = file.readlines()

my_aws_access_key_id = lines[0].strip()
my_aws_secret_access_key = lines[1].strip()
my_aws_session_token = lines[2].strip()
huggingface_cli_token = lines[3].strip()

In [ ]:
# from IPython.display import clear_output
# huggingface_cli_token = input("INSERT YOUR huggingface-cli_token:")
# clear_output()

In [ ]:
!huggingface-cli login --token $huggingface_cli_token

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
pip install -q boto3 --progress-bar off

In [ ]:
import boto3

# Inizializza il client DynamoDB con le credenziali manuali
dynamodb = boto3.resource(
    'dynamodb',
    region_name='eu-west-1',
    aws_access_key_id=my_aws_access_key_id,
    aws_secret_access_key=my_aws_secret_access_key,
    aws_session_token=my_aws_session_token
)

table = dynamodb.Table(table_name)

In [ ]:
# Funzione per aggiungere un nuovo record
def add_record(table, text_id):
    # Dati del nuovo record
    item = {
        'text_id': text_id  # Colonna e valore da inserire
        # Aggiungi qui altre colonne se necessario, ad esempio:
        # 'another_column': 'some_value'
    }

    # Aggiungi il record nella tabella
    try:
        table.put_item(Item=item)
        print(f"Record aggiunto con successo: {item}")
    except Exception as e:
        print(f"Errore durante l'inserimento del record: {e}")

# Funzione per estrarre tutti i valori della colonna 'text_id'
def extract_all_processed_text_ids(table):
    try:
        # Scansione iniziale della tabella
        response = table.scan()
        items = response['Items']

        # Lista per contenere i valori 'text_id'
        text_id_list = [item['text_id'] for item in items]

        # Gestione della paginazione (se ci sono più pagine di dati)
        while 'LastEvaluatedKey' in response:
            response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
            items = response['Items']
            text_id_list.extend([item['text_id'] for item in items])

        return text_id_list

    except Exception as e:
        print(f"Errore durante l'estrazione: {e}")
        #return []
        raise(e)

In [ ]:
processed_text_ids = extract_all_processed_text_ids(table)
print(f"Sono gia' stati processati {len(processed_text_ids)} testi")

Sono gia' stati processati 6395 testi


In [ ]:
import pickle

In [ ]:
# @title
import pandas as pd
data = pd.read_excel(input_data_path)

In [ ]:
import glob

def update_data(data_sel):

  processed_texts_from_dynamodb = extract_all_processed_text_ids(table)

  processed_texts_from_dynamodb = [int(x) for x in processed_texts_from_dynamodb]

  #print(processed_texts_from_dynamodb)

  data = data_sel[~data_sel.text_id.isin(processed_texts_from_dynamodb)].copy() # DEEP COPY

  print("Amount of processed files: ", len(processed_texts_from_dynamodb))

  return data.sample(frac=1.00)

# data_sel = update_data(data_sel)
# data_sel.shape[0]

In [ ]:
import torch
# from transformers import AutoModelForCausalLM, AutoTokenizer
torch.set_default_device("cuda")
!pip3 install -q -U bitsandbytes #==0.42.0
!pip3 install -q -U peft #==0.8.2
!pip3 install -q -U trl #==0.7.10
!pip3 install -q -U accelerate #==0.27.1
#!pip3 install -q -U datasets #==2.17.0
!pip3 install -q -U transformers #==4.38.1
token = huggingface_cli_token
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_4bit=True)

tokenizer = AutoTokenizer.from_pretrained(model_name, token = token)
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quantization_config,token = token,trust_remote_code=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.1/108.1 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.9/330.9 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 90.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 94.3 MB/s eta 0:00:00


tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.11G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

## Run the model

In [ ]:
# data_sel = data #.iloc[:3]
data_sel = data[data.text_id.isin([2758, 2978, 2932])] #.iloc[:3]

In [ ]:
# data_sel = update_data(data_sel)

In [ ]:
data_sel

,Unnamed: 0,text_id,text,sdg_true,sentiment,sdgwashing,sdgsmearing,sentiment_output,sdgwashing_output,sdgsmearing_output,...,sdgwashing_result_3,sdgwashing_result_4,sdgwashing_result_5,sdgwashing_result_6,sdgwashing_result_7,sdgsmearing_result_1,sdgsmearing_result_2,sdgsmearing_result_3,sdgsmearing_result_4,sdgsmearing_result_5
3116,3116,2978,"Over time, personalized technology will supers...",9,positive,NOT SDG washing,NOT SDG smearing,"['positive', 'The text talks about how persona...","['NOT SDG washing', ""The statement specificall...","['NOT SDG smearing', 'The statement positively...",...,NaN,NaN,NaN,NaN,NaN,The statement positively mentions an advanceme...,NaN,NaN,NaN,NaN
3636,3636,2932,Components of the program will include “pollut...,12,positive,NOT SDG washing,NOT SDG smearing,"['positive', 'The text discusses programs that...","['NOT SDG washing', 'This statement mentions a...","['NOT SDG smearing', ""The text provides a deta...",...,The text also mentions that The Netherlands ha...,NaN,NaN,NaN,NaN,The text provides a detailed and specific desc...,NaN,NaN,NaN,NaN
5638,5638,2758,This is also the reason why countries with sim...,1,positive,NOT SDG washing,NOT SDG smearing,"['positive', 'The text discusses the potential...","['NOT SDG washing', ""The text provides an in-d...","['NOT SDG smearing', 'This sentence provides a...",...,"However, the overall message encourages respon...",NaN,NaN,NaN,NaN,This sentence provides a thorough and specific...,NaN,NaN,NaN,NaN


In [ ]:
# Stop!

In [ ]:
results = []

# while 1:

  # data_sel = update_data(data_sel) # AGGIORNO IL DATAFRAME CON I DATI DA PROCESSARE A OGNI STEP

  # if data_sel.shape[0]==0:
  #   break


for index, row in data_sel.iterrows():

  cnt = index

  text_id = row['text_id']
  text = row["text"].replace('\n','').replace('{','(').replace('}',')')
  sdg_true = row["sdg_true"]

  file = open(log_filename, 'a+', encoding='utf-8')

  print("\n\n##################################################")
  file.write("\n\n##################################################\n")

  print("TEXT:", row["text"].replace('\n',''),'\n\n')
  file.write(f"TEXT: " + row['text'].replace('\n','') + "\n\n\n")

  template = """Given the following input text, between triple quotes, with its associated classification label with respect to one of the Sustainable Development Goals (SDGs), further classify the text with respect to the three labels defined below:

  - "positive": the text implies or explicitly states that what is affirmed or described leads to a significant advancement in favor of the goal indicated by the SDG under which it was previously classified.

  - "negative": the text implies or explicitly states that what is affirmed or described leads to a significant advancement against the goal indicated by the SDG under which it was previously classified.

  - "neutral": the text does not imply or explicitly state that what is affirmed or described leads to a significant advancement either in favor or against the goal indicated by the SDG under which it was previously classified.

Return the result in JSON format with 4 keys:

  - "label": The assigned label for the input text [this key/value pair is mandatory].

  - "explanation_1": A description of the reasoning behind the classification decision [this key/value pair is mandatory].

  - "explanation_2": Additional description of the reasoning behind the classification decision [this key/value pair is optional].

  - "explanation_3": Another additional description of the reasoning behind the classification decision [this key/value pair is optional].


The input text with its label are:
'''
INPUT TEXT: \""""+text+"""\"
SDG_CLASSIFICATION: "SDG-"""+str(sdg_true)+"""\"
OUTPUT:
'''
"""

  # template = "Ciao come stai?"

  #print(template)



  #model.generate(**encoded_input, pad_token_id=tokenizer.eos_token_id)

  input_ids = tokenizer(template, return_tensors="pt").to("cuda")

  # print(input_ids['input_ids'].shape[1])
  prompt_length = input_ids['input_ids'].shape[1]

  # raise("STOP")

  outputs = model.generate(**input_ids, max_new_tokens=100)

  output = tokenizer.decode(outputs[0][prompt_length:]).replace('<bos>','').replace('<eos>','')

  # inserire modulo di andrea per i json e farsi restituire un json...

  result_extended = (cnt, text_id, sdg_true, output)

  print(result_extended)
  file.write(str(result_extended)+'\n')

  with open(output_data_path+f'/sdg_classification_{text_id}.pkl', 'wb') as f:
      pickle.dump(result_extended, f)

  add_record(table, str(text_id)) # AGGIUNGE L'ID DEL RECORD PROCESSATO NELLA TABELLA DI DYNAMODB, IN MODO CHE GLI ALTRI WORKER NON LO RIPROCESSINO

  file.close()

    # break # PER FORZARE L'USCITA DAL CICLO INTERNO E PERMETTERE L'AGGIORNAMENTO DEL DATAFRAME



##################################################
TEXT: Over time, personalized technology will supersede one-size-fits-all models of education. A number of mathematics applications for smartphones and basic mobiles show learners how to solve questions step-by-step. For example, mobile apps now make it easy for teachers to administer short quizzes to test understanding instantaneously or synched with books - no paper, red pens or data entry are necessary. 




/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:452: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


(3116, 2978, 9, '```json\n{\n  "label": "positive",\n  "explanation_1": "The text suggests that personalized technology will enhance the effectiveness of education, aligning with SDG-9\'s target of improving education outcomes and access."\n  "explanation_2": "The mention of mobile apps replacing paper-based assessments aligns with SDG-9\'s focus on digitalization and innovation in education."\n}\n```\n\n**Note:** The provided input text is classified under SDG-9')
Record aggiunto con successo: {'text_id': '2978'}


##################################################
TEXT: Components of the program will include “pollution control in poor densely populated areas, environmentally sustainable development in poor urban areas, cleaner production in industry, sustainable livelihoods in and around marine protected areas, and capacity development support for environmental planning and management. The broad aim of these programs is poverty alleviation through approaches that build local capacity

In [ ]:
# !ls "/content/gdrive/MyDrive/Colab/SDG/data/../Llama-3/data/output/2024.05.09-02_Llama3_8B-Instruct_bfloat-16_3-few-shot"

In [ ]:
STOP!!!

In [ ]:
data_sel = data #.iloc[:3]

In [ ]:
import glob

files = glob.glob(output_data_path + "/*.pkl")

results = []
for file in files:
    res = pd.read_pickle(file)

    # try:
    #     if len(res[3]['choices'])>1:
    #         raise Exception('ERROR: Multilabel detected!')
    #         break
    #     sdg_pred = res[3]['choices'][0]['message']['content']
    #     res4df = (res[0], res[1], res[2], sdg_pred)
    #     print(res4df)
    #     tuple_list.append(res4df)
    # except:
    #     print("#################################################################")
    #     print(res)
    #     print("#################################################################")

    # print(res)
    results.append(res)
    # break

len(results)

In [ ]:
results_processed = pd.DataFrame(results, columns=['cnt', 'text_id', 'sdg_true', 'result'])
results_processed.head(3)

In [ ]:
#results_processed.iloc[0].result.split("Answer with label only usign the format:---LABEL---")[1]
# results_processed['result_short'] = results_processed['result'].apply(lambda x: x.split("Answer with label only usign the format:---LABEL---")[1])

In [ ]:
# results_processed['label'] = results_processed['result'].str.extract(r'SDG (\d+)')
# results_processed['label'] = results_processed['result'].str.extract(r'SDG-(\d+)')
# results_processed['label'] = results_processed['result_short'].str.extract(r'SDG (\d+)')

results_processed['label'] = results_processed['result'].str.extract(r'SDG (\d+)')
tmp_df_1 = results_processed[~results_processed.label.isna()]
# tmp_df_1

results_processed['label'] = results_processed['result'].str.extract(r'SDG-(\d+)')
tmp_df_2 = results_processed[~results_processed.label.isna()]
# tmp_df_2
# tmp_df_1 = results_processed['result'].str.extract(r'SDG-(\d+)')

results_processed = pd.concat([tmp_df_1,tmp_df_2])
results_processed

# results_processed['result'].str.extract(r'SDG-(\d+)')[~results_processed.label.isna()]

# results_processed.head(3)

In [ ]:
# results_processed[results_processed.text_id==6144]

In [ ]:
# results_processed[results_processed.label.isna()] #.str.extract(r'---SDG (\d+)---')

In [ ]:
# results_processed[results_processed.label.isna()]

In [ ]:
final = data_sel.merge(results_processed, left_on='text_id', right_on='text_id')
final.head(3)

In [ ]:
text_ids_with_errors = final[final.label.isna()].text_id.to_list()
list(set(text_ids_with_errors))

In [ ]:
final  = final[~final.label.isna()]

In [ ]:
import numpy as np
from sklearn.metrics import classification_report

y_true = final.sdg
y_pred = final.label
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

In [ ]:
# classification_report_filename = output_data_path+f'/{experiment_name}_classification_report.xlsx'
# classification_report_filename

In [ ]:
# report=classification_report(y_true, y_pred, output_dict=True)
# report_df = pd.DataFrame(report).transpose()
# report_df.to_excel(classification_report_filename)

In [ ]:
# test = pd.read_excel(classification_report_filename)
# test

In [ ]:
results_processed_merged = final

In [ ]:
# CORREZIONI NEL CASO IN CUI IL MODELLO RESTITUISCA DUPLICATI O OUTPUT NON UTILI PER DEFINIRE LA CLASSE O CLASSI NON PREVISTE

import random

# NEL CASO IN CUI IL MODELLO RESTITUISCA DUPLICATI SELEZIONO SEMPRE IL PRIMO
results_processed_merged = results_processed_merged.drop_duplicates(subset='text_id', keep='first')

# # NEL CASO IN CUI IL MODELLO RESTITUISCA OUTPUT NON UTILI PER DEFINIRE LA CLASSE SI APPLICA LA CLASSE 0
# # results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'predicted_label_clean'] = 0
# results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'predicted_label_clean'] = results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'sdg'] - 1

# # NEL CASO IN CUI IL MODELLO RESTITUISCA CLASSI NON PREVISTE SI APPLICA LA CLASSE 0
# results_processed_merged['predicted_label_clean'] = results_processed_merged.predicted_label_clean.apply(lambda x: 0 if int(x)>16 else x)

# Lista di valori sdg_list
sdg_list = [str(x) for x in range(17)]
# sdg_list = [x for x in range(17)]
# print(sdg_list)

# Se il valore di 'predicted_label_clean' e' NaN o un valore numerico non presente nella lista sdg_list
# allora assegna a  'predicted_label_clean' un valore qualunque estratto a caso da sdg_list ma diverso dal valore di 'sdg' per la riga in esame
# Modifica della colonna 'predicted_label_clean' secondo le regole specificate
for index, row in results_processed_merged.iterrows():
    if pd.isna(row['label']) or row['label'] not in sdg_list:
#         new_value = random.choice([x for x in sdg_list if x != row['sdg']])
        results_processed_merged.at[index, 'label'] = 0  # new_value

In [ ]:
results_processed_merged.head(3)

In [ ]:
import numpy as np
from sklearn.metrics import classification_report

y_true = results_processed_merged.sdg
y_pred = results_processed_merged.label
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

In [ ]:
output_datapath = output_data_path

In [ ]:
classification_report_filename = output_datapath+f'/{experiment_name}_classification_report.xlsx'
classification_report_filename

report=classification_report(y_true, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df[['precision', 'recall', 'f1-score']] *= 100
report_df.to_excel(classification_report_filename)

In [ ]:
report_df

In [ ]:
accuracy = report_df.loc['accuracy']['precision']
precision = report_df.loc['macro avg']['precision']
recall = report_df.loc['macro avg']['recall']
f1_score = report_df.loc['macro avg']['f1-score']

In [ ]:
print(precision, recall, accuracy, f1_score)

In [ ]:
print(round(precision, 1), round(recall, 1), round(accuracy, 1), round(f1_score, 1))